In [1]:
#loading libraries
%matplotlib inline
import os
from skimage import io
import imageio
import numpy as np
import skimage.transform
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize

In [4]:
cwd = os.getcwd()
print("current working directory is %s" %(cwd))

paths    = {"/Tr20",
            "/Tr20-valiSets"
           }
imgsize  = [78, 68]
use_gray = 1

compress_name = "splitDataset"
#converttograyifincasenotchanged
def rgb2gray(rgb):
    if len(rgb.shape) is 3:
        return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])
    else:
        return rgb

current working directory is /Users/uttam/Dropbox/sandbox-stuffs/college paper-2018_sep/Ai_final works/AI_codebase


In [5]:
nclass     = len(paths)
valid_exts = [".jpg",".png",".jpeg"]
img_count  = 0
for i, relpath in zip(range(nclass), paths):
    path = cwd + "/" + relpath
    flist = os.listdir(path)
    for f in flist:
        if os.path.splitext(f)[1].lower() not in valid_exts:
            continue
        fullpath = os.path.join(path, f)
        currimg  = imageio.imread(fullpath)
        # Converttograyscale  
        if use_gray:
            grayimg  = rgb2gray(currimg)
        else:
            grayimg  = currimg
        # we will reshapping array
        graysmall = skimage.transform.resize(grayimg, [imgsize[0], imgsize[1]])/255
        grayvec   = np.reshape(graysmall, (1, -1))
        # Save 
        curr_label = np.eye(nclass, nclass)[i:i+1, :]
        if img_count is 0:
            totalimg   = grayvec
            totallabel = curr_label
        else:
            totalimg   = np.concatenate((totalimg, grayvec), axis=0)
            totallabel = np.concatenate((totallabel, curr_label), axis=0)
        img_count    = img_count + 1
print ("Total number of images are %d " % (img_count))

/usr/local/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Total number of images are 1500 


In [6]:
#we will split diferent datas into train and test for validations
def img_gen_shape(string, x):
    print ("split %s is %s " % (string, x.shape,))
    
randidx    = np.random.randint(img_count, size=img_count)
trainidx   = randidx[0:int(3*img_count/5)]
testidx    = randidx[int(3*img_count/5):img_count]

training_imdata   = totalimg[trainidx, :]
training_labeldata = totallabel[trainidx, :]
testing_imdata    = totalimg[testidx, :]
testing_labeldata  = totallabel[testidx, :]

img_gen_shape("training imges", training_imdata)
img_gen_shape("training label", training_labeldata)
img_gen_shape("testing img", testing_imdata)
img_gen_shape("testing label",testing_labeldata) 


savepath = cwd + "/compressed_datasets/" + compress_name + ".npz"
np.savez(savepath, training_imdata=training_imdata, training_labeldata=training_labeldata 
         , testing_imdata=testing_imdata, testing_labeldata=testing_labeldata, imgsize=imgsize, use_gray=use_gray)
print ("Succesfully Saved to %s" % (savepath))
print ("Array compressed")

split training imges is (900, 5304) 
split training label is (900, 2) 
split testing img is (600, 5304) 
split testing label is (600, 2) 
Succesfully Saved to /Users/uttam/Dropbox/sandbox-stuffs/college paper-2018_sep/Ai_final works/AI_codebase/compressed_datasets/splitDataset.npz
Array compressed


In [7]:
#importing Important libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

/usr/local/Cellar/python/3.7.2_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [8]:
#loadingcompressedDatasetsfromdirectory
cwd = os.getcwd()
loadpath = cwd + "/compressed_datasets/splitDataset.npz"

showP = np.load(loadpath)
training_imdata = showP['training_imdata']
training_labeldata = showP['training_labeldata']
testing_imdata    = showP['testing_imdata']
testing_labeldata  = showP['testing_labeldata']
ntrain = training_imdata.shape[0]
nclass = training_labeldata.shape[1]
dim    = training_imdata.shape[1]
ntest  = testing_imdata.shape[0]
imgsize = showP['imgsize']
print ("%d train images loaded" % (ntrain))
print ("%d test images loaded" % (ntest))
print ("%d dimensional input" % (dim))
print ("Image size is %s" % (imgsize))
print ("%d classes" % (nclass))

900 train images loaded
600 test images loaded
5304 dimensional input
Image size is [78 68]
2 classes


In [9]:
tf.set_random_seed(0)

#valuesForrunningSteps
learning_rate = 0.200
train_Epochs = 100
batch_size  = ntrain
noOfstep    = 20

#hiddenNeuronsDefining
n_hidden_1 = 20 #1stlayerinNUmfeatures
n_hidden_2 = 20 #2ndlayerinNumfeatures
n_input    = dim # data input 
n_classes  = nclass # total classes (0-9 digits)

#tensorValueinputs
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

#mainneuralNetworks
def neural_networkModel(_X, _weights, _biases):
    layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _weights['wt1']), _biases['bia1'])) 
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _weights['wt2']), _biases['bia2']))
    return tf.matmul(layer_2, _weights['out']) + _biases['out']
    
#definingOurweightsandBiases
stddev = 0.4  
weights = {
    'wt1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'wt2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'bia1': tf.Variable(tf.random_normal([n_hidden_1])),
    'bia2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

pred = neural_networkModel(x, weights, biases)

#variableInitializing
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y)) 
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

#variableInitializing
init = tf.global_variables_initializer()

In [10]:
sess = tf.Session()
sess.run(init)

#ourtrainingsteps
for epoch in range(train_Epochs):
    avg_cost = 0
    total_batch = int(ntrain/batch_size)
    
    for i in range(total_batch):
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = training_imdata[randidx, :]
        batch_ys = training_labeldata[randidx, :]   
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        #calculateAverageloss
        avg_cost += sess.run(cost, 
                feed_dict={x: batch_xs, y: batch_ys})/total_batch
        #printingtheresults
    if epoch % noOfstep  == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, train_Epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testing_imdata, y: testing_labeldata})
        print (" Test accuracy: %.3f" % (test_acc))

print("accuracy test finished")

Epoch: 000/100 cost: 8.914620399
 Training accuracy: 0.288
 Test accuracy: 0.277
Epoch: 020/100 cost: 0.605227590
 Training accuracy: 0.712
 Test accuracy: 0.723
Epoch: 040/100 cost: 0.618607759
 Training accuracy: 0.689
 Test accuracy: 0.727
Epoch: 060/100 cost: 0.593586981
 Training accuracy: 0.720
 Test accuracy: 0.733
Epoch: 080/100 cost: 0.572857916
 Training accuracy: 0.737
 Test accuracy: 0.733
accuracy test finished
